# Imports

In [1]:
# TODO: write 100s of sentences which must contain at least 1 ambiguous word, perplexity on bigram/trigram
# TODO: train disambiguation model, add function to slt.languages.text.Urdu().
# TODO: then train language model to write more sentences which are disambiguated already

# WORD_SENSE_DISAMBIGUATION training pipeline (semi supervised)

# train word2vec with SignTokenizer on entire data
# label sentences for word-sense-disambiguation
# initialize disambiguated words with ambiguous form's vectors
# finetune just them (truncate to window size?)

# TODO: Filter Wikipedia data by perplexity by training on others and evaluating on Wikipedia
# TODO: make buckets and sample from them (more perp, less sampling)

In [ ]:

# ? parallel corpus:
    # PSL & urdu(transcribe)
    # PSL(T5-summarizer) & urdu(translate)
    # english(translate-verified)
    # random(translate-google)

# ? Vision:
    # MP.load("path")
    # MP.load("path", format="video")
    # MP.load("path", format="blaze_pose")
    # MP.from_blaze_pose("path")
    # MP.from_video("path")
    # MP.save("path")
    # MP.to(format="numpy")
    # MP.to_torch
    # MP.to_moviepy ?
    # MP.to_video(engine = "plt" or "gan")

In [1]:
SLD = "/Users/mudassar.iqbal/Library/CloudStorage/GoogleDrive-mdsriqb@gmail.com/My Drive/sign-language-translator/sign-language-datasets"
import re
import json
from collections import Counter
import pandas as pd
from tqdm.auto import tqdm

from importlib import reload
import sign_language_translator as slt
reload(slt)
# slt.set_dataset_dir(SLD)

from sign_language_translator.languages import vocab
from sign_language_translator.languages.text import urdu
from sign_language_translator.languages.sign import pakistan_sign_language
from sign_language_translator.models.language_models import mixer, beam_sampling, ngram_language_model
from sign_language_translator.models import ConcatenativeSynthesis
from sign_language_translator.text import metrics
from sign_language_translator.text import utils

reload(vocab), reload(urdu), reload(pakistan_sign_language), reload(metrics)
reload(mixer), reload(beam_sampling), reload(ngram_language_model), reload(utils)
None

In [2]:
ur = urdu.Urdu()
psl = pakistan_sign_language.PakistanSignLanguage()
t2s = ConcatenativeSynthesis("ur", "psl", "mp")

# Load Data

In [ ]:
with open("../model_training/ur_lm_training.txt", 'r') as f:
    data = f.read()
data = [ur.tokenize(d) for d in data]

In [11]:
with open("../model_training/token_to_id.json") as f:
    t2i = json.load(f)

# Train

In [13]:
model_sizes = [1,2,3,4,5,6]
slms = [
    ngram_language_model.NgramLanguageModel(window_size=w, unknown_token="<unk>", name=f"token-sw-w{w}")
    for w in model_sizes
]
[lm.fit(data) for lm in tqdm(slms, leave=True)]
mix = mixer.MixerLM(slms, [m**1.5 for m in model_sizes], name="ur-supported-token-unambiguous-mixed-w1-w6-lm")
sampler = beam_sampling.BeamSampling(mix, beam_width=3, start_of_sequence_token = "<", max_length=69)
mix

  0%|          | 0/6 [00:00<?, ?it/s]

'sign_language_translator.models.language_models.mixer.MixerLM'

Mixer LM: name="ur-supported-token-unambiguous-mixed-w1-w6-lm", unk_tok="<unk>"[6]
├── Ngram LM: name="token-sw-w1", unk_tok="<unk>", window=1, params=14174 | prob=2.3%
├── Ngram LM: name="token-sw-w2", unk_tok="<unk>", window=2, params=120596 | prob=6.6%
├── Ngram LM: name="token-sw-w3", unk_tok="<unk>", window=3, params=186896 | prob=12.1%
├── Ngram LM: name="token-sw-w4", unk_tok="<unk>", window=4, params=291644 | prob=18.6%
├── Ngram LM: name="token-sw-w5", unk_tok="<unk>", window=5, params=250857 | prob=26.1%
└── Ngram LM: name="token-sw-w6", unk_tok="<unk>", window=6, params=199543 | prob=34.3%

In [14]:
mix.save("ur-supported-token-unambiguous-mixed-ngram-w1-w6-lm.pkl")

In [16]:
mux = mixer.MixerLM.load("ur-supported-token-unambiguous-mixed-ngram-w1-w6-lm.pkl")
samplur = beam_sampling.BeamSampling(mux, beam_width=3, start_of_sequence_token = "<", max_length=69)


In [131]:
completion, log_prob = sampler(['<'])
print(ur.detokenize(completion))
print(t2s.translate(ur.detokenize(completion)))

<شوکت نے آگے میری ماں کو دیکھا۔۔۔>
['pk-hfad-1_ش', 'pk-hfad-1_o(single-handed-letter)', 'pk-hfad-1_k(single-handed-letter)', 'pk-hfad-1_ت', 'pk-hfad-1_نے', 'pk-hfad-1_اگلا', 'pk-hfad-1_میری', 'pk-hfad-1_ماں', 'pk-hfad-1_کو', 'pk-hfad-1_دیکھا']


In [27]:
completion, log_prob = samplur(['<'])
print(ur.detokenize(completion))
print(t2s.translate(ur.detokenize(completion)))

<وہ ایک ایک دن سال سال کا اک اک لفظ سے نفرت؛ جواب اور ابا کو کچھ اور نہ تھی۔>
['pk-hfad-1_وہ', 'pk-hfad-1_1', 'pk-hfad-1_1', 'pk-hfad-1_دن', 'pk-hfad-1_سال', 'pk-hfad-1_سال', 'pk-hfad-1_کا', 'pk-hfad-1_1', 'pk-hfad-1_1', 'pk-hfad-1_لفظ', 'pk-hfad-1_سے', 'pk-hfad-1_نفرت', 'pk-hfad-1_a(double-handed-letter)', 'pk-hfad-1_اور', 'pk-hfad-1_باپ', 'pk-hfad-1_کو', 'pk-hfad-1_چند', 'pk-hfad-1_اور', 'pk-hfad-1_نہیں', 'pk-hfad-1_تھی']


In [35]:
mixx.next_all("< وہ اپنی جگہ ".split())

([' ', '؛', '!', '۔', '،', '؟'],
 [0.9443084455324341,
  0.020807833537331663,
  0.0030599755201958334,
  0.015299877600979164,
  0.015911872705018332,
  0.0006119951040391665])

# `Write` to file

In [148]:
# get word senses
with open(SLD + '/text_corpora/temp/urdu_corpus_mini.txt', 'r')  as f:
    corpus = f.read().splitlines()
    tokenized_corpus = [ur.tokenize(l) for l in corpus]

[
    (line, [item for item in ur.get_word_senses(line) if item])
    for line in tokenized_corpus
    if set(line) & ambiguous_words
][:5]

[]

# evaluate written sentences

In [ ]:
with open(f"{slt.Settings.DATASET_ROOT_DIRECTORY}/text_corpora/temp/urdu_corpus_mini.txt") as f:
    lines = f.read().splitlines()

tokens = [[(tok+"(i)" if tok =="میں" else tok+"(came)" if tok in ["آئی", "آیا"] else tok+"(what)" if tok == 'کیا' else tok+"(we)" if tok=="ہم"else tok ) for tok in ur.tokenize(ur.preprocess(line))] for line in lines]
tags   = [ur.get_tags(tok) for tok in tokens]
signs  = [psl(tokens[i], tags[i]) for i in range(len(tokens))]
labels = [[d["signs"][0][0] for d in sent] for sent in signs ]

perp = metrics.Perplexity({tok for sent in labels for tok in sent})
perp.update_frequencies(labels)
[(perp.evaluate(l), ln) for l, ln in zip(labels, lines) if perp.evaluate(l) > 25]

[(25.245825875536944, 'آپ آؤ۔'),
 (25.33333333333333, 'آیا تھا؟'),
 (34.70700667011836, 'اپنا(ذاتی) نام تھا۔'),
 (25.33333333333333, 'تم آئے۔'),
 (28.985262713066014, 'میں آئی۔'),
 (26.920683752626147, 'میں آیا تھا۔'),
 (31.144349719464508, 'میں کام آؤں؟'),
 (38.300399931601696, 'ہم آئے۔')]